In [1]:
%pip install azure-ai-inference[opentelemetry]
%pip install azure-search-documents
%pip install azure-identity
%pip install openai

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_api-1.36.0-py3-none-any.whl.metadata (1.5 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)
Using cached opentelemetry_api-1.36.0-py3-none-any.whl (65 kB)

   ---- -----------------------------------  1/10 [isodate]
   ------------ ---------------------------  3/10 [charset_normalizer]
   ------------------------ ---------------  6/10 [opentelemetry-api]
   ---------------------------- -----------  7/10 [azure-core]
   ---------------------------- -----------  7/10 [azure-core]
   ------------------------------------ ---  9/10 [azure-ai-inferen

In [6]:
import os
import dotenv
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import VectorizedQuery, VectorizableTextQuery

# Carga las variables del archivo .env al entorno del sistema
dotenv.load_dotenv()

# Lee las variables del entorno
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME")

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")


openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
)

def get_embedding(text):
    return openai_client.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
        input=text
    ).data[0].embedding

In [7]:
user_question = "What is included in my Northwind Health Plus plan that is not in standard?"
user_question_vector = get_embedding(user_question)
print(user_question_vector)

[-0.0004826383665204048, -0.007185375317931175, 0.008878480643033981, -0.02254721149802208, -0.032788436859846115, 0.029760116711258888, -0.02271239273250103, -0.024543149396777153, -0.02183142676949501, -0.0006417971453629434, -0.004353208467364311, 0.02228567562997341, -0.031164156273007393, -0.005901780910789967, 0.00643173698335886, -0.0030369223095476627, 0.02262980304658413, -0.007281730882823467, 0.018541572615504265, -0.023731010034680367, -0.0009635560563765466, 0.010743650607764721, 0.018114853650331497, -0.008162696845829487, 0.0016019119648262858, 0.0034670811146497726, 0.0010538893984630704, -0.008761477656662464, -0.013145658187568188, 0.0020355121232569218, 0.04212116450071335, 0.008589413948357105, -0.009773211553692818, -0.023414412513375282, 0.00199077557772398, 0.017550485208630562, 0.0020802486687898636, -0.006097933277487755, 0.02890668250620365, 0.002023467794060707, 0.0241714920848608, 0.004201792646199465, 0.011239193379878998, 0.005423444323241711, -0.000979902

In [8]:
search_results = search_client.search(
    None,
    top=3,
    vector_queries=[
        VectorizableTextQuery( 
            text=user_question, k_nearest_neighbors=3, fields="text_vector"
        )
    ],
)

for result in search_results:
    print("Chunk ID:", result["chunk_id"])
    print("Title:", result["title"])
    print("Text:", result["chunk"])
    print()

Chunk ID: 36071692be7f_aHR0cHM6Ly9zdG9yYWdlcmFnMzE0MDYuYmxvYi5jb3JlLndpbmRvd3MubmV0L3JhZy1kb2N1bWVudHMvTm9ydGh3aW5kX0hlYWx0aF9QbHVzX0JlbmVmaXRzX0RldGFpbHMucGRm0_pages_121
Title: Northwind_Health_Plus_Benefits_Details.pdf
Text: their chronic conditions.  



• Be proactive: Talk to your doctor if you have any questions or concerns about your 

condition or care plan.  

At Northwind Health, we understand that managing chronic conditions can be challenging. 

That’s why we’re committed to helping our members get the care and support they need to 

stay healthy and active. Through our Chronic Condition Management Program, we provide 

members with access to an interdisciplinary team of healthcare professionals who can 

provide personalized care and support. We also offer a variety of resources and programs to 

help members manage their chronic conditions. With Northwind Health Plus, you can rest 

assured that you’ll have the support and resources you need to stay healthy and active. 



In [9]:
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME="gpt-4o"

context = ""
for result in search_results:
    context += result["chunk"] + "\n\n"

SYSTEM_MESSAGE = f"""
You are an AI Assistant.
Be brief in your answers. Answer ONLY with the facts listed in the retrieved text.

Context:
{context}
"""

USER_MESSAGE = user_question

response = openai_client.chat.completions.create(
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

Northwind Health Plus includes coverage for alternative therapies (e.g., acupuncture, chiropractic, massage therapy), a gym membership reimbursement, and comprehensive vision care (including coverage for glasses and contact lenses). These features are not included in the standard plan.
